In [1]:
import os

from ndbc_api import NdbcApi
import pandas as pd

REFRESH = False #Set to True to remake all files.
LATITUDES = 41.37754, 42.09690 #Vickery and Amherstburg
LONGITUDES = -83.51131, -82.67909 #Toledo and Kellys Island
START_DATE, END_DATE = '2015-01-01', '2025-01-01'
FILES = {
    'station_detail': 'Erie Western Basin Stations.csv',
    'station_data_roster': 'Erie Western Basin Station Data Roster.csv',
    'ocean_data': 'Erie Western Basin Ocean Data.csv'
    }
api = NdbcApi()

if REFRESH:
    for i in FILES.items():
        os.remove(i)

try:
    stations = pd.read_csv(FILES['station_detail'])
except FileNotFoundError:
    stations = (
        api
        .stations()
        .loc[
            lambda x: x.Lat.between(*LATITUDES) & x.Lon.between(*LONGITUDES),
            :
            ]
        .set_index('Station')
        )
    assert stations.index.is_unique, 'Station IDs are not unique.'
    stations.to_csv(FILES['station_detail'])

stations

,Lat,Lon,Elevation,Name,Owner,Program,Type,Includes Meteorology,Includes Currents,Includes Water Quality,DART Program
Station,,,,,,,,,,,
45165,41.702,-83.261,174.0,"Toledo Water Intake Buoy, Oregon, OH",Limno Tech,IOOS Partners,buoy,False,False,False,False
45200,41.724,-83.370,174.0,Maumee Bay Buoy,Cleveland Water Alliance,IOOS Partners,buoy,False,False,False,False
45201,41.601,-82.781,174.0,Erie Islands Buoy,Cleveland Water Alliance,IOOS Partners,buoy,False,False,False,False
45202,41.532,-82.941,174.0,Port Clinton Buoy,Cleveland Water Alliance,IOOS Partners,buoy,False,False,False,False
cmpo1,41.547,-83.015,175.0,"Camp Perry, OH",NWS Eastern Region,IOOS Partners,fixed,True,False,False,False
mrho1,41.544,-82.731,176.0,"9063079 - Marblehead, OH",NOS,NOS/CO-OPS,fixed,True,False,False,False
sbio1,41.629,-82.841,176.8,"South Bass Island, OH",NDBC,NDBC Meteorological/Ocean,fixed,True,False,False,False
thlo1,41.826,-83.194,173.5,Toledo Light No. 2 OH,GLERL,IOOS Partners,fixed,True,False,False,False
thro1,41.694,-83.473,176.2,"9063085 - Toledo, OH",NOS,NOS/CO-OPS,fixed,True,False,False,False


In [ ]:
def create_data_roster() -> pd.DataFrame:
    station_datas = {}

    for idx in stations.index:
        try:
            station_datas[idx] = list(
                api
                .available_historical(idx)
                .keys()
                )
        except Exception:
            print(f'Error getting data for station {idx}')
            continue
    
    columns = set().union(station_datas.items())
    data_roster = pd.DataFrame(
        data={
            k: [True if c in v else False for c in columns]
            for k, v in station_datas
            },
        columns=columns)


try:
    data_roster = pd.read_csv(FILES['station_data_roster'])
except FileNotFoundError:
    data_roster = create_data_roster()
    data_roster.to_csv(FILES['station_data_roster'])

data_roster

In [ ]:
OCEAN_DATA_FILE = 'Erie Western Basin Ocean Data.csv'

try:
    ocean_data = pd.read_csv(OCEAN_DATA_FILE)
except FileNotFoundError:
    ocean_data_stations = []
    
    for station in stations.Station.unique():
        

    ocean_data = api.get_data(
        station_ids=list(stations.Station.unique()),
        mode='ocean',
        )

,Station,Lat,Lon,Elevation,Name,Owner,Program,Type,Includes Meteorology,Includes Currents,Includes Water Quality,DART Program
286,45165,41.702,-83.261,174.0,"Toledo Water Intake Buoy, Oregon, OH",Limno Tech,IOOS Partners,buoy,False,False,False,False
310,45200,41.724,-83.370,174.0,Maumee Bay Buoy,Cleveland Water Alliance,IOOS Partners,buoy,False,False,False,False
311,45201,41.601,-82.781,174.0,Erie Islands Buoy,Cleveland Water Alliance,IOOS Partners,buoy,False,False,False,False
312,45202,41.532,-82.941,174.0,Port Clinton Buoy,Cleveland Water Alliance,IOOS Partners,buoy,False,False,False,False
722,cmpo1,41.547,-83.015,175.0,"Camp Perry, OH",NWS Eastern Region,IOOS Partners,fixed,True,False,False,False
1014,mrho1,41.544,-82.731,176.0,"9063079 - Marblehead, OH",NOS,NOS/CO-OPS,fixed,True,False,False,False
1195,sbio1,41.629,-82.841,176.8,"South Bass Island, OH",NDBC,NDBC Meteorological/Ocean,fixed,True,False,False,False
1270,thlo1,41.826,-83.194,173.5,Toledo Light No. 2 OH,GLERL,IOOS Partners,fixed,True,False,False,False
1272,thro1,41.694,-83.473,176.2,"9063085 - Toledo, OH",NOS,NOS/CO-OPS,fixed,True,False,False,False
1289,twco1,41.699,-83.259,173.5,"Toledo Crib, OH",Limno Tech,IOOS Partners,fixed,True,False,False,False
